## 协程

In [1]:
def A():
    print('1')
    print('2')
    print('3')

def B():
    print('x')
    print('y')
    print('z')


In [3]:
def consumer():
    r = ''
    while True:
        n = yield r
        if not n:
            return 
        print('[CONSUMER] Consuming %s...' % n)
        r = '200 OK'


def produce(c):
    c.send(None)
    n = 0
    while n < 5:
        n = n + 1
        print('[PRODUCER] Producing %s...' % n)
        r = c.send(n)
        print('[PRODUCER] Cosumer return: %s' %r)
    c.close()


c = consumer()
produce(c)

[PRODUCER] Producing 1...
[CONSUMER] Consuming 1...
[PRODUCER] Cosumer return: 200 OK
[PRODUCER] Producing 2...
[CONSUMER] Consuming 2...
[PRODUCER] Cosumer return: 200 OK
[PRODUCER] Producing 3...
[CONSUMER] Consuming 3...
[PRODUCER] Cosumer return: 200 OK
[PRODUCER] Producing 4...
[CONSUMER] Consuming 4...
[PRODUCER] Cosumer return: 200 OK
[PRODUCER] Producing 5...
[CONSUMER] Consuming 5...
[PRODUCER] Cosumer return: 200 OK


## asyncio

In [1]:
import asyncio

@asyncio.coroutine
def hello():
    print("Hello world!")
    # 异步调用asyncio.sleep(1):
    r = yield from asyncio.sleep(1)
    print("Hello again!")

# 获取EventLoop:
loop = asyncio.get_event_loop()
# 执行coroutine
loop.run_until_complete(hello())
loop.close()

RuntimeError: This event loop is already running

Hello world!
Hello again!


In [2]:
import threading
import asyncio

@asyncio.coroutine
def hello():
    print('Hello world! (%s)' % threading.currentThread())
    yield from asyncio.sleep(1)
    print('Hello again! (%s)' % threading.currentThread())

loop = asyncio.get_event_loop()
tasks = [hello(), hello()]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

RuntimeError: This event loop is already running

Hello world! (<_MainThread(MainThread, started 4557637056)>)
Hello world! (<_MainThread(MainThread, started 4557637056)>)
Hello again! (<_MainThread(MainThread, started 4557637056)>)
Hello again! (<_MainThread(MainThread, started 4557637056)>)


In [3]:
import asyncio

@asyncio.coroutine
def wget(host):
    print('wget %s...' % host)
    connect = asyncio.open_connection(host, 80)
    reader, writer = yield from connect
    header = 'GET / HTTP/1.0\r\nHost: %s\r\n\r\n' % host
    writer.write(header.encode('utf-8'))
    yield from writer.drain()
    while True:
        line = yield from reader.readline()
        if line == b'\r\n':
            break
        print('%s header > %s' % (host, line.decode('utf-8').rstrip()))
    # Ignore the body, close the socket
    writer.close()

loop = asyncio.get_event_loop()
tasks = [wget(host) for host in ['www.sina.com.cn', 'www.sohu.com', 'www.163.com']]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

RuntimeError: This event loop is already running

wget www.sohu.com...
wget www.163.com...
wget www.sina.com.cn...
www.sohu.com header > HTTP/1.1 200 OK
www.sohu.com header > Content-Type: text/html;charset=UTF-8
www.sohu.com header > Connection: close
www.sohu.com header > Server: nginx
www.sohu.com header > Date: Sun, 23 Jun 2019 13:55:31 GMT
www.sohu.com header > Cache-Control: max-age=60
www.sohu.com header > X-From-Sohu: X-SRC-Cached
www.sohu.com header > Content-Encoding: gzip
www.sohu.com header > FSS-Cache: HIT from 3811884.5122614.5650496
www.sohu.com header > FSS-Proxy: Powered by 5777994.9054804.7616636
www.163.com header > HTTP/1.1 302 Moved Temporarily
www.163.com header > Date: Sun, 23 Jun 2019 13:56:22 GMT
www.163.com header > Content-Length: 0
www.163.com header > Connection: close
www.163.com header > Server: Cdn Cache Server V2.0
www.163.com header > Location: http://www.163.com/special/0077jt/error_isp.html
www.163.com header > X-Via: 1.0 PSsccdyd2se53:2 (Cdn Cache Server V2.0)
www.sina.com.cn header > HTTP/1.1 302 

## async/await

In [ ]:
@asyncio.coroutine
def hello():
    print("Hello world!")
    r = yield from asyncio.sleep(1)
    print("Hello again!")

In [ ]:
async def hello():
    print("Hello world!")
    r = await asyncio.sleep(1)
    print("Hello again!")